In [1]:
from keras.applications.vgg16 import VGG16
import tensorflow as tf
import numpy as np
import imageio

# VGG16 모델 로드
model = VGG16(weights='imagenet', include_top=False)

for layer in model.layers:
    # Checks for a convolutional layer
    if 'conv' not in layer.name:
        continue
    print(f'Layer name: {layer.name}, output_shape: {layer.output.shape}')

# 그래디언트 상승 함수 정의
def gradient_ascent(n_times, input_img, step):
    for _ in range(n_times):
        with tf.GradientTape() as tape:
            tape.watch(input_img)
            layer_output = model.get_layer(layer_name).output
            model_output = tf.keras.Model(inputs=model.inputs, outputs=layer_output)
            layer_output_value = model_output(input_img)
            loss = tf.reduce_mean(layer_output_value[:, :, :, filter_index])
        grads = tape.gradient(loss, input_img)
        grads /= (tf.sqrt(tf.reduce_mean(tf.square(grads))) + 1e-5)
        input_img.assign_add(grads * step)

# 이미지 후처리 함수 정의
def deprocess_image(x):
    x = x - x.mean()
    x = x / (x.std() + 1e-5)
    x = x * 0.1
    x = x + 0.5
    x = np.clip(x, 0, 1)
    x = x * 255
    x = x.astype('uint8')
    return x

# 필터 활성화를 위한 레이어와 필터 인덱스 설정
layer_name = 'block3_conv1'
filter_index = 127

# 입력 이미지 크기 정의
img_width = 224
img_height = 224

# 랜덤 이미지로 필터 활성화 시도
input_img_data = tf.Variable(np.random.random((1, img_width, img_height, 3)) * 20 + 128, dtype=tf.float32)

step = 0.1
n_times = 20
gradient_ascent(n_times, input_img_data, step)
result_img = input_img_data.numpy()[0]
result_img = deprocess_image(result_img)
imageio.imwrite('%s_filter_%d.png' % (layer_name, filter_index), result_img)

print("이미지 생성 완료.")

Layer name: block1_conv1, output_shape: (None, None, None, 64)
Layer name: block1_conv2, output_shape: (None, None, None, 64)
Layer name: block2_conv1, output_shape: (None, None, None, 128)
Layer name: block2_conv2, output_shape: (None, None, None, 128)
Layer name: block3_conv1, output_shape: (None, None, None, 256)
Layer name: block3_conv2, output_shape: (None, None, None, 256)
Layer name: block3_conv3, output_shape: (None, None, None, 256)
Layer name: block4_conv1, output_shape: (None, None, None, 512)
Layer name: block4_conv2, output_shape: (None, None, None, 512)
Layer name: block4_conv3, output_shape: (None, None, None, 512)
Layer name: block5_conv1, output_shape: (None, None, None, 512)
Layer name: block5_conv2, output_shape: (None, None, None, 512)
Layer name: block5_conv3, output_shape: (None, None, None, 512)
이미지 생성 완료.


### tensorflow.Variable

#### Ex1

In [3]:
import tensorflow as tf

# tf.Variable을 사용하여 변수를 생성
my_var = tf.Variable([1.0, 2.0, 3.0], dtype=tf.float32)

# 변수의 값을 출력
print("Initial value:", my_var.numpy())

# 변수의 값을 수정
my_var.assign([4.0, 5.0, 6.0])
print("Updated value:", my_var.numpy())

# 변수의 값을 증가시킴
my_var.assign_add([1.0, 1.0, 1.0])
print("Incremented value:", my_var.numpy())

# 변수의 값을 감소시킴
my_var.assign_sub([2.0, 2.0, 2.0])
print("Decreased value:", my_var.numpy())

Initial value: [1. 2. 3.]
Updated value: [4. 5. 6.]
Incremented value: [5. 6. 7.]
Decreased value: [3. 4. 5.]


#### Ex2

In [6]:
import tensorflow as tf

# 학습 가능한 파라미터를 tf.Variable로 정의
W = tf.Variable(tf.random.normal([3, 2]), name='weight')
print("Initial weights:", W.numpy())
b = tf.Variable(tf.zeros([2]), name='bias')
print("Initial bias:", b.numpy())

# 간단한 선형 모델 정의
def model(X):
    return tf.matmul(X, W) + b

# 손실 함수 정의 (MSE)
def loss_fn(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

# 옵티마이저 정의
optimizer = tf.optimizers.legacy.SGD(learning_rate=0.01)

# 입력 데이터 및 레이블
X = tf.constant([[1.0, 2.0, 3.0]], dtype=tf.float32)
y_true = tf.constant([[1.0, 2.0]], dtype=tf.float32)

# 그래디언트 테이프를 사용하여 학습
with tf.GradientTape() as tape:
    y_pred = model(X)
    loss = loss_fn(y_true, y_pred)

# 그래디언트 계산 및 적용
grads = tape.gradient(loss, [W, b])
optimizer.apply_gradients(zip(grads, [W, b]))

print("Updated weights:", W.numpy())
print("Updated bias:", b.numpy())

Initial weights: [[-0.06313162  0.54826355]
 [ 0.5222686  -0.85945195]
 [-0.07272832  0.26974446]]
Initial bias: [0. 0.]
Updated weights: [[-0.06076383  0.5718776 ]
 [ 0.5270042  -0.8122238 ]
 [-0.06562494  0.34058666]]
Updated bias: [0.00236779 0.02361407]
